# Time series forecasting

This tutorial uses safeds on **US inflation rates data** to predict future inflation with a recurrent neural network. The **US inflation rates data** is a time series. A time series, is a series with frequently measured values. This time series measures the inflation of the US every Month from January 1947 until June 2023.


1. Load your data into a `Table`. The data is available under `docs/tutorials/data/US_Inflation_rates.csv`:


In [ ]:
from safeds.data.tabular.containers import Table

inflation = Table.from_csv_file("data/US_Inflation_rates.csv")
# For visualisation purposes we only print out the first 15 rows.
inflation.slice_rows(start=0,length=15)

This dataset contains two columns: date and value. The date column is right now still a string type with a format like this: "Year-Month-Day". We can convert it into a temporal type column like this:

In [ ]:
inflation = inflation.replace_column("date", [inflation.get_column("date").transform(lambda cell:cell.str.to_date())])
inflation.slice_rows(start=0,length=15)

Let's have a look on the dataset.

In [ ]:
inflation.plot.line_plot(x_name="date", y_names=["value"])

2. In the second step we prepare the data, to train our neural network. For that we need to normalize our data, because neural networks work better on small values. After that, we split the data into a training and test set.


In [ ]:
from safeds.data.tabular.transformation import StandardScaler

scaler = StandardScaler()
trained_scaler, inflation_scaled = scaler.fit_and_transform(inflation)
train_set, test_set = inflation_scaled.split_rows(percentage_in_first=0.9, shuffle=False)

3. After the data preparation we can start defining our neural network. We do this by defining the different layers and their output size.

In [ ]:
from safeds.ml.nn import NeuralNetworkRegressor

from safeds.ml.nn.converters import (
    InputConversionTimeSeries,
)
from safeds.ml.nn.layers import (
    ForwardLayer,
    LSTMLayer,
)

neural_network = NeuralNetworkRegressor(
    InputConversionTimeSeries(),
    [LSTMLayer(256), ForwardLayer(1, "none")],
)

4. When working with time series data, the data is segmented into windows prior to being fed into the neural network. Each window consists of a sequence of consecutive data points. Windowing data is beneficial for neural networks as it allows them to more effectively learn the relationships between data points. In Safe-DS, the windowing step is automated; users simply need to classify their dataset as a time series, as demonstrated below.

In [ ]:
from safeds.data.labeled.containers import TimeSeriesDataset

fitted_neural_network = neural_network.fit(
    TimeSeriesDataset(
        train_set,
        "value",
        window_size=12,
        forecast_horizon=1,
        continuous=False,
        extra_names= ["date"]
    ),
    epoch_count=25
)

5. Now that we defined and trained our model, we can start making predictions.

In [ ]:

prediction = fitted_neural_network.predict(test_set)
prediction = prediction.to_table()

6. Now we only need to inverse our predictions and we can start visualizing them:


In [ ]:
prediction = trained_scaler.inverse_transform(prediction)
test_set = trained_scaler.inverse_transform(test_set)

In [ ]:
preds_with_test = prediction.add_columns([test_set.slice_rows(start=13).rename_column("value","true_value").get_column("true_value")])

In [ ]:
preds_with_test.plot.line_plot("date", ["value", "true_value"])